現在は、目的のコメントが付与されたgit履歴において
<br>正規表現を用いて全ての diff --git ~~ （ファイル名）
<br>を取得しているため
<br>コメントが付与されたファイルを知ることができない
<br>そのため、変更点に対してfor文で処理をし、
<br>どのファイルが目的のファイルか取得するような処理を行う

In [100]:
import pandas as pd
import os, re
from IPython.display import display
from tqdm import tqdm

In [101]:
saikyo = pd.read_csv('./saikyo-data.csv')
saikyo.head(1)

,id,project,path,revision,url,link,start,end,isLicense,isAutoGenerated,isIncludedKeyword,satdKeyword,isIncludedOurKeyword,ourKeyword,isSelected,comment,below,class
0,1,31z4/storm-docker,31z4/storm-docker/1.1.3/Dockerfile\n31z4/storm...,664a7f4730f58be00fd7fe0526d2c13180107c6f,https://github.com/31z4/storm-docker/blob/664a...,https://github.com/31z4/storm-docker/blob/664a...,7\n7\n7,7\n7\n7,0,0,0,NaN,0,NaN,0.0,# Add a user with an explicit UID/GID and crea...,RUN set -eux; \\nRUN set -eux; \\nRUN set -eux; \,NaN


In [102]:
results = os.listdir('./result')

## diff で目的のファイルを取り出す作業

コメントが含まれ、かつ追加されたコメントであること

### 気づいたこと
SATDの起源を探すのは難しい
コメントが変化していたら探せなくなる

今回は「最初のコメントが追加されたら？」っていう条件で「目的コメントが追加されたとき」を探していたけど
<br>最初のコメントはそもそも前の段階で追加されており、途中コメントだけ変化していたら、上の条件を満たすことはない
<br>よって「目的コメントが追加されたファイルがない」という事態になっていた。
<br>しょうがないため、今回目的のコメントが完成した瞬間に着目して抽出してみる

In [103]:
output_dir = os.listdir("./preprocessed/")

In [104]:
for i, result in enumerate(results):
    
    if result[-4:] != ".csv":
        continue
        
    ## 一度実行したものは除外
    if result in output_dir:
        continue
    
    id = result.split('_')[0]
    comments = saikyo[saikyo["id"]==int(id)]["comment"].values[0].splitlines()
    print("\n".join(comments))

    df = pd.read_csv(f'./result/{result}', index_col=0)
    df.insert(value=0, loc=len(df.columns)-2, column="diff_files")
    df.fillna(0, inplace=True)
    
    idxlist = df[df["diff"] != 0].index.values.tolist()

    for idx in idxlist:
        lines = df.loc[idx, "diff"].splitlines()
        files = []
        notdiff_files = []
        
        ## diff で変更点をあげているが
        ## 本当にSATDが追加されたのか？どのファイルに追加されたのか？がわからないため
        ## 追加されたときそのファイル名を保存 それ以外はただの変更コミットとしてファイル名を保存しておく
        for l in lines:
            if l[0:10] == "diff --git":
                tmpfile = l[11:]
                notdiff_files.append(tmpfile)

            try:
                ## どれか一つでも
                ## コメントが含まれる & 追加された行であること を条件
                for comment in comments:
                    if ((comment in l) and (l[0] == "+")):
                        files.append(tmpfile)
                        notdiff_files.remove(tmpfile)
                        del tmpfile
                        break
            except NameError:
                continue
        
        try:
            del tmpfile
        except:
            pass
        
        df.loc[idx, "diff_files"] = str(files)
        df.loc[idx, "file"] = str(notdiff_files)
        
    display(df.head(3))
    df.to_csv(f'./preprocessed/{result}')

# https://wiki.php.net/rfc/argon2_password_hash (7.2+)


,commitid,date,file,diff_files,diff,rename
0,a3cf31c9020559777c731d980047450cba744122,"[('Thu May 14 12:46:13 2020 +0000', '')]",['a/7.3/alpine3.10/cli/Dockerfile b/7.3/alpine...,0,0,[]
1,057b438e69093c927a84cce4308c7ad08ccdd5b0,"[('Thu May 14 12:46:13 2020 +0000', '')]",['a/7.2/alpine3.10/cli/Dockerfile b/7.2/alpine...,0,0,[]
2,8c6d1f42185d2a3cd22f381ba4bcd95ec52fa31f,"[('Thu May 14 12:46:13 2020 +0000', '')]",['a/7.4/alpine3.10/cli/Dockerfile b/7.4/alpine...,0,0,[]


# Copy ansible playbooks


,commitid,date,file,diff_files,diff,rename


# Bail out early if NODE_VERSION is not provided


...

,commitid,date,file,diff_files,diff,rename
0,29b163923699c18c6d76a95df10ba52665ef8741,"[('Wed May 27 10:22:38 2020 -0700', '')]",[],0,0,[]
1,5f111eef86fe44b85f7db613497372fc54eeeee9,"[('Tue May 26 12:03:43 2020 -0700', '')]",['a/3.8-rc/alpine/Dockerfile b/3.8-rc/alpine/D...,0,0,[]
2,13d43346f3c0b4de9a15c9589ecdf8b3e22b689c,"[('Mon May 25 17:05:18 2020 +0000', '')]",['a/3.8/alpine/Dockerfile b/3.8/alpine/Dockerf...,[],13d43346f3c0b4de9a15c9589ecdf8b3e22b689c\nAuth...,[]


# Check that Erlang/OTP crypto & ssl were compiled against OpenSSL correctly


,commitid,date,file,diff_files,diff,rename
0,29b163923699c18c6d76a95df10ba52665ef8741,"[('Wed May 27 10:22:38 2020 -0700', '')]",[],0,0,[]
1,5f111eef86fe44b85f7db613497372fc54eeeee9,"[('Tue May 26 12:03:43 2020 -0700', '')]",['a/3.8-rc/alpine/Dockerfile b/3.8-rc/alpine/D...,0,0,[]
2,13d43346f3c0b4de9a15c9589ecdf8b3e22b689c,"[('Mon May 25 17:05:18 2020 +0000', '')]",['a/3.8/alpine/Dockerfile b/3.8/alpine/Dockerf...,0,0,[]


# DO NOT EDIT: created by update.sh from Dockerfile-debian.template


,commitid,date,file,diff_files,diff,rename
0,ba1500b05ee8889c99a46f1d0de63ab40d3e2926,"[('Sat May 23 22:34:08 2020 +0000', '')]",['a/19.0-rc/apache/Dockerfile b/19.0-rc/apache...,[],ba1500b05ee8889c99a46f1d0de63ab40d3e2926\nAuth...,[]
1,ccdf46609ff8419ffd7c5ce4e51a117e378b72b6,"[('Fri May 22 14:33:39 2020 +0200', '')]",[],0,0,[]
2,8335bf86182512ca98c6a12de0d5a92c56324d93,"[('Thu May 21 14:00:18 2020 +0000', '')]",['a/19.0-rc/apache/Dockerfile b/19.0-rc/apache...,0,0,[]


# はじめに全てのファイルに対してrename 処理を行う

In [149]:
output_dir = os.listdir("./preprocessed_rename/")

for i, result in enumerate(results):
    
    print(f'============= {i}/{len(results)}')
    print(result)
    
    if result != "433_mongo.csv":
        continue

    if result[-4:] != ".csv":
        continue
        
    ## 一度実行したものは除外
    if result in output_dir:
        continue
        
    df = pd.read_csv(f'./preprocessed/{result}', index_col=0)
        
    renamefiles = ','.join(df["rename"].values.tolist())
    renamefiles = re.findall(r'\(\'([^\)]*)\'\)', renamefiles)[::-1]
    
    ## 辞書に変換するときにエラーが出た
    ## 2433_cntk で以下のrenameが発生したため
    ## "BrainScript/BrainScript--extending the CNTK config language, Frank Seide August 2015.pptx', 'Source/CNTK/BrainScript/Doc/BrainScript--extending the CNTK config language, Frank Seide August 2015.pptx"
    try:
        renamefiles = dict(map(lambda rename: rename.replace('\'', '').split(', '), renamefiles))
    except ValueError:
        renamefiles = list(map(lambda rename: rename.replace('\'', '').split(', '), renamefiles))
        for i in renamefiles:
            if len(i) == 2:
                pass
            else:## Error: 2つ以上のペアがでたら削除
                renamefiles.remove(i)
        renamefiles = dict(renamefiles)
        
    renamepairs = {"before":[], "after":[]}
#     print(renamefiles)
    
    ## before の list を
    ## 同じ indexの after に変換する
    for before, after in renamefiles.items():
        ## 抵抗してみました
        ## Dockerに関する名前が絶対にあるという強い自信
        if "Dockerfile" in before:
            pass
        elif "dockerfile" in before:
            pass
        elif "docker" in before:
            pass
        elif "Docker" in before:
            pass
        else:
            continue
            
        try:
            idx = renamepairs["after"].index(before)
            renamepairs["after"][idx] = after
            renamepairs["before"][idx].append(before)
        except ValueError:
            renamepairs["after"].append(after)
            renamepairs["before"].append([before])

    print(renamepairs)
    for num  in range(len(renamepairs["after"])):
        ## rename before -> after
        before = '|'.join(renamepairs["before"][num])
        before = f'(a/|b/)({before})'
        after = 'a/'+renamepairs["after"][num]

        df["file"] = df["file"].apply(lambda s: re.sub(before, after, str(s)))
        df["diff_files"] = df["diff_files"].apply(lambda s: re.sub(before, after, str(s)))
    break
    display(df.head(3))
    df.to_csv(f'./preprocessed_rename/{result}')    

============= 0/409
2141_ghost_nondebt.csv
============= 1/409
1612_dockerfiles.csv
============= 2/409
508_php_nondebt.csv
============= 3/409
2387_AIforEarth-API-Development_nondebt.csv
============= 4/409
1507_docker-splunk_nondebt.csv
============= 5/409
1588_dockerfiles.csv
============= 6/409
1877_batch-shipyard_nondebt.csv
============= 7/409
269_docker.csv
============= 8/409
1424_rocker_nondebt.csv
============= 9/409
1159_docker-nuxeo_nondebt.csv
============= 10/409
737_compose.csv
============= 11/409
1733_mediawiki-docker.csv
============= 12/409
2433_cntk_nondebt.csv
============= 13/409
463_openjdk.csv
============= 14/409
2915_PowerShell-Docker_nondebt.csv
============= 15/409
2032_bosh-deployment-resource_nondebt.csv
============= 16/409
702_tomcat_nondebt.csv
============= 17/409
2790_newman_nondebt.csv
============= 18/409
2460_cntk.csv
============= 19/409
472_openjdk_nondebt.csv
============= 20/409
944_mq-container_nondebt.csv
============= 21/409
3095_php-zendser